https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE130353

In [1]:
from __init__ import *

In [2]:
gse_id = "GSE130353"

In [3]:
path_template = "{}{}_{{}}_x_sample.tsv".format(SETTING["directory_path"], gse_id)

In [4]:
_x_sample_ = kraft.geo.get_gse(
    gse_id, SETTING["directory_path"], overwrite=SETTING["overwrite"]
)

/media/kwat/CarrotCake/garden/data/chronic_fatigue_syndrome/GSE130353_family.soft.gz exists.
PLATFORM = GPL18573
SAMPLE = GSM3736559
SAMPLE = GSM3736560
SAMPLE = GSM3736561
SAMPLE = GSM3736562
SAMPLE = GSM3736563
SAMPLE = GSM3736564
SAMPLE = GSM3736565
SAMPLE = GSM3736566
SAMPLE = GSM3736567
SAMPLE = GSM3736568
SAMPLE = GSM3736569
SAMPLE = GSM3736570
SAMPLE = GSM3736571
SAMPLE = GSM3736572
SAMPLE = GSM3736573
SAMPLE = GSM3736574
SAMPLE = GSM3736575
SAMPLE = GSM3736576
SAMPLE = GSM3736577
SAMPLE = GSM3736578
SAMPLE = GSM3736579
SAMPLE = GSM3736580
SAMPLE = GSM3736581
SAMPLE = GSM3736582
SAMPLE = GSM3736583
SAMPLE = GSM3736584
SAMPLE = GSM3736585
SAMPLE = GSM3736586
SAMPLE = GSM3736587
SAMPLE = GSM3736588
SAMPLE = GSM3736589
SAMPLE = GSM3736590
SAMPLE = GSM3736591
SAMPLE = GSM3736592
SAMPLE = GSM3736593
SAMPLE = GSM3736594
SAMPLE = GSM3736595
SAMPLE = GSM3736596
SAMPLE = GSM3736597
SAMPLE = GSM3736598
--------------------------------------------------------------------------------
      

In [5]:
binary_feature_x_sample = _x_sample_[2]

binary_feature_x_sample.to_csv(path_template.format("binary_feature"), sep="\t")

binary_feature_x_sample

,HC7,HC13,HC16,HC19,HC20,HC22,HC24,HC25,HC39,HC41,...,PQ4,PQ6,PQ8,PQ9,PQ12,PQ14,PQ15,PQ17,PQ20,PQ21
Feature (binary),,,,,,,,,,,,,,,,,,,,,
donor id.Donor_1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
donor id.Donor_2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
donor id.Donor_3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
donor id.Donor_4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
donor id.Donor_5,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
donor id.Donor_6,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
donor id.Donor_7,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
donor id.Donor_8,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
donor id.Donor_9,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
file_path = kraft.internet.download(
    "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE130353&format=file",
    SETTING["directory_path"],
    name="{}_RAW.tar".format(gse_id),
    overwrite=SETTING["overwrite"],
)

/media/kwat/CarrotCake/garden/data/chronic_fatigue_syndrome/GSE130353_RAW.tar exists.


In [7]:
sample_ = []

with tarfile.open(file_path) as io:

    for tarinfo in io:

        transcript_n_ = pd.read_csv(
            io.extractfile(tarinfo),
            compression="gzip",
            skiprows=1,
            sep="\t",
            index_col=0,
        ).iloc[:, 6]

        transcript_n_.name = tarinfo.name.split(sep="_")[1]

        sample_.append(transcript_n_)

gene_x_sample = pd.DataFrame(data=sample_).T

gene_x_sample.index = kraft.name_biology.name_gene(gene_x_sample.index.to_numpy())

gene_x_sample.index.name = "Gene"

gene_x_sample = kraft.pd.collapse(gene_x_sample)

gene_x_sample.to_csv(path_template.format("gene"), sep="\t")

gene_x_sample

Named 36581/56625 (64.60%)
(56625, 40)
Collapsing...
(36581, 40)


,HC7,HC13,HC16,HC19,HC20,HC22,HC24,HC25,HC39,HC41,...,PQ4,PQ6,PQ8,PQ9,PQ12,PQ14,PQ15,PQ17,PQ20,PQ21
Gene,,,,,,,,,,,,,,,,,,,,,
A1BG,8,9,17,5,14,14,18,12,16,9,...,11,16,14,1,2,10,15,10,13,14
A1BG-AS1,26,31,54,24,31,27,76,38,22,17,...,28,73,45,7,3,11,56,20,55,17
A1CF,0,0,2,0,2,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
A2M,37,26,64,17,80,58,36,29,41,40,...,18,98,50,31,13,29,74,43,56,48
A2M-AS1,64,50,114,36,119,86,77,58,52,42,...,47,175,100,33,24,43,120,58,109,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11AP1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
ZYG11B,1044,1056,1474,918,1115,1068,1413,984,1043,787,...,1091,2187,1749,291,156,558,1468,1085,1513,637
ZYX,355,570,1790,248,918,323,1084,797,296,267,...,790,1184,998,96,45,147,769,324,2020,204
